# ETL (Extract Transform Load)

Aqui vamos a ***extraer*** los datos de la api de `Messari API` para obtener series de tiempo de precios de criptomonedas y luego ***transformar*** esos datos en un formato que sea útil para el análisis. Finalmente, ***cargar*** esos datos en un archivo `json` para su posterior análisis.

## Importamos las librerias

In [1]:
from _utils import *

## Usamos la `API` de **Messari** para obtener data en intervalos de tiempo de los halvings

Obtenemos las monedas mediante un endpoint del plan gratuito

In [2]:
coins_list = fetch_data(api=APIS.MESSARI,uri="marketdata/v1/assets/ath", parameters={})

In [3]:
coins_list = coins_list["data"]

In [4]:
len(coins_list)

15455

Tambien cargamos el dataset de `coins_info` para utilizar la informacion del `market_cap`, esta info ya esta limpia(notebook `eda_coins_exchanges/2_clean_dataset.ipynb`) por lo que solo se encuentran monedas con `market_cap` mayores a `0.0` y no nulos 

In [5]:
coins_info = read_json("../../datasets/json/coins.json")

In [6]:
len(coins_info)

7862

Ahora obtenemos los names en base a `low_cap_coins` y `high_cap_coins` para filtrar `coins_list`

In [7]:
# Creamos el set para los `names` 
coins_info_names = set()
list(map(lambda x: coins_info_names.add(x["name"]), coins_info))
# Obtenemos el tamaño 
len(coins_info_names)

3391

Ahora que tenemos `low_coins_info_names` y `high_coins_info_names` podemos filtrar `coins_list` para obtener `low_coins_list` y `high_coins_list`

In [8]:
coins = list(filter(lambda x: x["name"] in coins_info_names, coins_list))
len(coins)

2126

Usamos el endpoint `marketdata/v1/assets/{assetId}/price/time-series` para obtener la datafr los 4 halvings

In [11]:
def get_marketdata_per_halving(days:int, interval:str, coins: list)->dict[str,list]:
    halving_data: dict[str,list] = {}
    for halving in HALVINGS_DATE:
        print(f"halving {halving.value} ")
        halving_data[halving.value] = [] 
        start_timestamp, end_timestamp = get_start_end_timestamps(halving.value, days)
        print(f"star: {start_timestamp}, end: {end_timestamp}")
        for coin in coins:
            new_coin = coin.copy()
            endpoint = f"marketdata/v1/assets/{coin["id"]}/price/time-series?interval={interval}&startTime={start_timestamp}&endTime={end_timestamp}"
            response = fetch_data(APIS.MESSARI, endpoint, parameters={})
            new_coin["halving"] = response["data"]
            halving_data[halving.value].append(new_coin)
    return halving_data

In [12]:
halving_data: dict[str,list] = get_marketdata_per_halving(DAYS_INTERVAL, "1d", coins)
save_json(halving_data, "./metadata/halving.json")

halving 2/12/2012 
star: 1354424400, end: 1376024400
halving 2/07/2016 
star: 1467435600, end: 1489035600
halving 3/05/2020 
star: 1588482000, end: 1610082000
halving 20/04/2024 
star: 1713589200, end: 1735189200
